In [ ]:
import pandas as pd
import numpy as np
import sys, os
# ceļu norādīt nevajag, ja pakotne ir instalēta
documentpath=os.getcwd()
libpath=os.path.abspath(os.path.join(documentpath, os.pardir))
sys.path.append(libpath)
from sudrabainiemakoni.cloudimage import CloudImage
from sudrabainiemakoni import plots

### Katalogi un failu nosaukumi

In [ ]:
# katalogs ar piemēra failiem
sample_directory = 'SampleData'
# katalogs ar rezultātu failiem
results_directory = 'SampleResults'
if not os.path.exists(results_directory):
    os.makedirs(results_directory)
# sudrabaino mākoņu attēls
case_id = 'js_202106210100'
filename_jpg = f'{sample_directory}/{case_id}.jpg'
# nociparotās zvaigznes
filename_stars = f'{sample_directory}/{case_id}_zvaigznes.txt'
# ģeogrāfiskais garums platums
lat, lon = 56.693, 23.656

### Inicializācija, datuma, ģeogrāfisko koordināšu un zvaigžņu uzstādīšana

In [ ]:
#inicializējam CloudImage tipa objektu
cldim = CloudImage(case_id, filename_jpg)

In [ ]:
#Norādam datumu, pieprasot to no EXIF
cldim.setDateFromExif()

In [ ]:
#Uzstādām novērotāja ģeogrāfisko platumu un garumu (grādos)
cldim.setLocation(lat=lat, lon=lon)

In [ ]:
print('UTC:', cldim.date)
print(cldim.location.to_geodetic())

In [ ]:
# uzstādām zvaigžņu sarakstu
df = pd.read_csv(filename_stars, sep='\t', header=None)
# zvaigžņu nosaukumi pirmajā kolonā
starnames = df[0]
# atbilstošās pikseļu koordinātes otrajā un trešajā kolonā
pixels=np.array(df[[1,2]])
cldim.setStarReferences(starnames, pixels)

In [ ]:
# izdrukājam zvaigžņu ekvatoriālās un pikseļu koordinātes pārbaudes nolūkos
print(cldim.getSkyCoords())
print(cldim.getPixelCoords())

### Kameras kalibrēšana

In [ ]:
# kalibrējam kameras novietojumu, izdrukājam referencēšanas kļudu pikseļos (divas pēdējas rindiņas)
cldim.PrepareCamera()

In [ ]:
az, el, rot = cldim.camera.get_azimuth_elevation_rotation()
print(f'Kameras ass azimuts {az:.2f}°') 
print(f'Kameras ass augstums virs horizonta {el:.2f}°')
print(f'Kameras pagrieziena leņķis {rot:.2f}°')

In [ ]:
# noglabājam kameras iestatījumus
cldim.SaveCamera(f'{results_directory}/{case_id}_kam.json')

In [ ]:
# noglabājam dotajam attēlam atbilstošo projektu - tas saturēs norādi uz attēlu, kameras, zvaigznes, novērotāja pozīciju, koordinātu sistēmas 
# šo failu var vēlāk ielasīt ar CloudImage.load
cldim.save(f'{results_directory}/{case_id}.proj')

### Attēls ar horizontālo koordinātu režģi

In [ ]:
#plots.PlotAltAzGrid(cldim,   stars = True, showplot=True, from_camera = True)
# ja attēls jānoglabā failā, pie parametriem jāpievieno outImageDir = results_directory+'/'

### Attēls ar horizontālo koordinātu režģi  
Ar x simboliem uzzīmētas zvaigžņu pozīcijas atbilstoši kameras referencēšanai, ar o simboliem, atbilstoši uzdotajām pikseļu koordinātēm  
Ideālā gadījumā tie sakrīt, bet jāņem vērā arī tas, ka pārējā skata laukā koordināšu režģis nedrīkst būt izkropļots, jābauda arī horizonta pozīcija

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20,10))
plots.PlotAltAzGrid(cldim,   stars = False, showplot=False, from_camera = True, ax=ax)
# zvaigžņu koordinātes enu sistēmā, vienības attālumam
enu_unit_coords = cldim.get_stars_enu_unit_coords()
# zvaigžņu pikseļu koordinātes atbilstoši referencētai kamerai
campx=cldim.camera.camera_enu.imageFromSpace(enu_unit_coords)
# ievadītās zvaigžņu pikseļu koordinātes
pxls = cldim.getPixelCoords()
for sr, cpx in zip(cldim.starReferences, campx):
    ix, iy = sr.pixelcoords
    p=ax.plot(ix,iy, marker='o', fillstyle='none')
    ax.annotate(sr.name, xy=(ix,iy), xytext=(3,3), color='#AAFFAA', fontsize=16, textcoords='offset pixels')
    ax.plot(cpx[0],cpx[1], marker='x', fillstyle='none', color=p[0].get_color())
fig.savefig(f'{results_directory}/{case_id}_horizkoord.jpg', dpi=300, bbox_inches='tight')
plt.show()

### Attēls ar ekvatoriālo koordināšu režģi

In [ ]:
# iegūstam attēla WCS ekvatoriālajām kordinātēm
cldim.GetWCS(sip_degree=2, fit_parameters={'projection':'TAN'})

In [ ]:
plots.PlotRADecGrid(cldim,   stars = False, showplot=True )
# ja attēls jānoglabā failā, pie parametriem  jāpievieno outImageDir = results_directory+'/'